In [2]:
import pandas as pd
from tqdm import tqdm
import seaborn as sns

In [2]:
def augustus_refs(df):
    out_df= pd.DataFrame()
    accessions=df["accession"].unique().tolist()
    for acc in accessions:    
        busco = pd.read_csv(f"/ebio/abt6_projects7/dliso/dlis/leon_pipeline/analysis/sqanti/ref_genes/augustus_mappings/busco_{acc}.txt", header=None)
        house = pd.read_csv(f"/ebio/abt6_projects7/dliso/dlis/leon_pipeline/analysis/sqanti/ref_genes/augustus_mappings/housekeeping_{acc}.txt", header=None)
        nlr = pd.read_csv(f"/ebio/abt6_projects7/dliso/dlis/leon_pipeline/analysis/sqanti/ref_genes/augustus_mappings/nlrs_{acc}.txt", header=None)

        busco["type"] = "busco"
        house["type"] = "housekeeping"
        nlr["type"] = "nlr"
        gene_type = pd.concat([busco,house,nlr])
        gene_type.columns = ["ref_isoform","type"]
        gene_type = gene_type.groupby("ref_isoform")["type"].apply(', '.join).reset_index()

        gene_type["ref_gene"] = gene_type["ref_isoform"].str.split(".").str[0]
        gene_type.loc[gene_type["type"] == "busco, nlr"] = "nlr"
        gene_type.loc[gene_type["type"] == "housekeeping, nlr"] = "nlr"
        gene_type.loc[gene_type["type"] == "busco, housekeeping"] = "housekeeping"

        sub=df.loc[df["accession"]==acc]
        sub = pd.merge(left=sub, right=gene_type, how='left', left_on='associated_gene', right_on='ref_gene')

        
        out_df=pd.concat([out_df,sub])
    return out_df

## Access TAMA)

In [5]:
ref=pd.read_pickle("all_runs/tama_augustus_sqanti.pkl")

In [7]:
ref["structural_category"].value_counts()

full-splice_match          1760959
novel_not_in_catalog        825013
incomplete-splice_match     419668
novel_in_catalog            206821
genic                       166785
antisense                    29343
fusion                       28951
intergenic                   22284
genic_intron                     8
Name: structural_category, dtype: int64

In [8]:
# load full df
ref=pd.read_pickle("all_runs/tama_augustus_sqanti.pkl")
# add new column
ref["cat_assign"]="None"
# and fill new colum based on 4 categories
ref.loc[(ref["structural_category"] != "antisense") & (ref["structural_category"] != "fusion") & (ref["structural_category"] != "intergenic"), "cat_assign"]="one2one"
ref.loc[(ref["structural_category"] == "antisense"), "cat_assign"]="antisense"
ref.loc[(ref["structural_category"] == "fusion"), "cat_assign"]="fusion"
ref.loc[((ref["structural_category"] == "intergenic") | (ref["structural_category"] == "genic_intron")), "cat_assign"]="novel"

# in some cases, fusion transcripts were reported as one2one if they had perfect splice junctions over their fusion genes.
# filter them and add replace the assignment
ref["association"] = ref["associated_gene"].str.split("_").str.len()
ref.loc[(ref["cat_assign"]=="one2one")&(ref["association"]>1), "cat_assign"] = "fusion"

# add gene read support
# cat_read_support is the number of reads supporting the individual category assignments of a gene (can be different in terms of clear, fusion and antisense)
# gene_association_read_support: sometimes there are multiple clear assignments within a gene, caused by a fusion gene also associated with the reads covering
# multiple ref genes. This number gives the support for each clear-case gene in the reference.
ref["cat_read_support"]=ref.groupby(by=["cat_assign","accession","gene"])["isoform_read_support"].transform("sum")
ref["gene_association_read_support"]=ref.groupby(by=["cat_assign","accession","gene","associated_gene"])["isoform_read_support"].transform("sum")

## Prepare Fusion genes for both type assignment and later detailed analysis (fusion type)

In [4]:
def augustus_refs_fusion(df):
    out_df= pd.DataFrame()
    accessions=df["accession"].unique().tolist()
    for acc in accessions:    
        busco = pd.read_csv(f"/ebio/abt6_projects7/dliso/dlis/leon_pipeline/analysis/sqanti/ref_genes/augustus_mappings/busco_{acc}.txt", header=None)
        house = pd.read_csv(f"/ebio/abt6_projects7/dliso/dlis/leon_pipeline/analysis/sqanti/ref_genes/augustus_mappings/housekeeping_{acc}.txt", header=None)
        nlr = pd.read_csv(f"/ebio/abt6_projects7/dliso/dlis/leon_pipeline/analysis/sqanti/ref_genes/augustus_mappings/nlrs_{acc}.txt", header=None)

        busco["type"] = "busco"
        house["type"] = "housekeeping"
        nlr["type"] = "nlr"
        
        gene_type = pd.concat([busco,house,nlr])
        gene_type.columns = ["ref_isoform","type"]
        gene_type = gene_type.groupby("ref_isoform")["type"].apply(', '.join).reset_index()

        gene_type["ref_gene"] = gene_type["ref_isoform"].str.split(".").str[0]
        gene_type.loc[gene_type["type"] == "busco, nlr"] = "nlr"
        gene_type.loc[gene_type["type"] == "housekeeping, nlr"] = "nlr"
        gene_type.loc[gene_type["type"] == "busco, housekeeping"] = "housekeeping"

        sub=df.loc[df["accession"]==acc]
        sub = pd.merge(left=sub, right=gene_type, how='left', left_on='gene_split', right_on='ref_gene')
        
        out_df=pd.concat([out_df,sub])
    return out_df

In [5]:
fus=ref.loc[ref["cat_assign"]=="fusion"].copy()
fus["gene_split"]=fus["associated_gene"].str.split("_")

In [6]:
split=fus.explode("gene_split")

In [7]:
# test for novel genes in fusion genes
for i in split["gene_split"].tolist():
    if i == "novelGene":
        print(i)

In [8]:
fusion_types=augustus_refs_fusion(split).drop(columns=["ref_isoform","ref_gene"])
fusion_types["type"] = fusion_types["type"].fillna("other")

In [9]:
fusion_types.reset_index(drop=True, inplace=True)

In [10]:
fusion_out=pd.DataFrame()
for i in fusion_types.groupby(by=["accession", "gene", "isoform"], as_index=False):
    t=i[1]["type"].value_counts()
    t=t.reset_index()

    t=t["index"].tolist()

    i[1]["fusion_order"] = "_".join(i[1]["type"].tolist())
    
    if "busco" in t:
        i[1]["busco"] = 1
    if "nlr" in t:
        i[1]["nlr"] = 1
    if "housekeeping" in t:
        i[1]["housekeeping"] = 1
    if "other" in t:
        i[1]["other"] = 1

    fusion_out=pd.concat([fusion_out,i[1]], axis=0)

In [11]:
fusion_out

,gene,isoform,accession,associated_gene,associated_transcript,exons,length,isoform_read_support,gene_read_support,structural_category,...,association,cat_read_support,gene_association_read_support,gene_split,type,fusion_order,other,busco,nlr,housekeeping
2299,G10006,G10006.5,at6923,g19413_g19414,novel,10,2461,1,11,fusion,...,2,1,1,g19413,other,other_other,1.0,NaN,NaN,NaN
2300,G10006,G10006.5,at6923,g19413_g19414,novel,10,2461,1,11,fusion,...,2,1,1,g19414,other,other_other,1.0,NaN,NaN,NaN
2301,G10013,G10013.3,at6923,g19424_g19425,novel,2,3552,1,5,fusion,...,2,2,2,g19424,other,other_other,1.0,NaN,NaN,NaN
2302,G10013,G10013.3,at6923,g19424_g19425,novel,2,3552,1,5,fusion,...,2,2,2,g19425,other,other_other,1.0,NaN,NaN,NaN
2303,G10013,G10013.5,at6923,g19424_g19425,novel,2,3534,1,5,fusion,...,2,2,2,g19424,other,other_other,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72219,G9951,G9951.9,col0,AT4G00030_AT4G00040,AT4G00030.1,3,2332,1,21,full-splice_match,...,2,19,19,AT4G00040,other,busco_other,1.0,1.0,NaN,NaN
72220,G9987,G9987.2,col0,AT4G00560_AT4G00570,novel,23,3761,1,71,fusion,...,2,1,1,AT4G00560,busco,busco_other,1.0,1.0,NaN,NaN
72221,G9987,G9987.2,col0,AT4G00560_AT4G00570,novel,23,3761,1,71,fusion,...,2,1,1,AT4G00570,other,busco_other,1.0,1.0,NaN,NaN
72222,G9989,G9989.11,col0,AT4G00630_AT4G00620,novel,8,1542,1,88,fusion,...,2,1,1,AT4G00630,other,other_other,1.0,NaN,NaN,NaN


In [12]:
fusion_types= fusion_out.drop(columns=["gene_split","type","association"]).drop_duplicates()
fusion_types.fillna({'other':0, 'busco':0, 'nlr':0,'housekeeping':0}, inplace=True)

## Connect all ref gene sets (Clear Cases + Fusion genes (Order: NLR>House>Busco>Other)) - This creates the DF for subsequent analysis

### The reason to do it this preciesely for the reference groups to have a fair comparison between them and add duplicate fusion genes to each of the potential groups it belongs. In the antisense case, it was done quick without distinguishing and just keeping the order above.

In [13]:
NLRs=pd.DataFrame()
for acc in ref["accession"].unique().tolist():
    
    clear_isos=ref.loc[(ref["accession"]==acc)&(ref["cat_assign"] == "one2one")]
    isos_type=augustus_refs(clear_isos).drop(columns=["ref_isoform","ref_gene"])
    nlr_isos=isos_type.loc[isos_type["type"]=="nlr"]

    
    b=fusion_types.loc[(fusion_types["nlr"]==1)&(fusion_types["accession"]==acc)]["isoform"].tolist()
    fusion_isos=ref.loc[(ref["accession"]==acc)&(ref["isoform"].isin(b))]

    c=pd.concat([nlr_isos,fusion_isos], ignore_index=True)
    
    c["isoforms_per_gene"]= c.groupby(by="gene")["isoform"].transform(lambda x: len(x.tolist()))
    c["ORFs_per_gene"]= c.groupby(by=["gene"])["Sqanti_ORF_seq"].transform(lambda x: len(set(x.tolist())))
    c["isoforms_per_ORFs"]= c.groupby(by=["gene","Sqanti_ORF_seq"])["isoform"].transform(lambda x: len(x.tolist()))
    
    #print("Mean nlr isoforms per gene:")
    hipg=c[["gene","isoforms_per_gene"]].drop_duplicates()
    #print(hipg["isoforms_per_gene"].apply("mean"))

    #print("Mean nlr ORFs per gene:")
    hopg=c[["gene","ORFs_per_gene"]].drop_duplicates()
    #print(hopg["ORFs_per_gene"].apply("mean"))
    #print("\n")
    
    
    NLRs=pd.concat([NLRs,c])
NLRs["type"]="nlr"

In [14]:
House=pd.DataFrame()
for acc in ref["accession"].unique().tolist():
    
    clear_isos=ref.loc[(ref["accession"]==acc)&(ref["cat_assign"] == "one2one")]
    isos_type=augustus_refs(clear_isos).drop(columns=["ref_isoform","ref_gene"])
    house_isos=isos_type.loc[isos_type["type"]=="housekeeping"]

    
    b=fusion_types.loc[(fusion_types["housekeeping"]==1)&(fusion_types["accession"]==acc)]["isoform"].tolist()
    fusion_isos=ref.loc[(ref["accession"]==acc)&(ref["isoform"].isin(b))]

    c=pd.concat([house_isos,fusion_isos], ignore_index=True)
    #c=ref.loc[(ref["accession"]==f"{acc}") & (ref["gene"].isin(set(a[f"{acc}"].tolist() + b[f"{acc}"].tolist())))&~(ref["cat_assign"].isin(["antisense","novel"]))].copy()
    
    c["isoforms_per_gene"]= c.groupby(by="gene")["isoform"].transform(lambda x: len(x.tolist()))
    c["ORFs_per_gene"]= c.groupby(by=["gene"])["Sqanti_ORF_seq"].transform(lambda x: len(set(x.tolist())))
    c["isoforms_per_ORFs"]= c.groupby(by=["gene","Sqanti_ORF_seq"])["isoform"].transform(lambda x: len(x.tolist()))
    
    #print("Mean nlr isoforms per gene:")
    hipg=c[["gene","isoforms_per_gene"]].drop_duplicates()
    #print(hipg["isoforms_per_gene"].apply("mean"))

    #print("Mean nlr ORFs per gene:")
    hopg=c[["gene","ORFs_per_gene"]].drop_duplicates()
    #print(hopg["ORFs_per_gene"].apply("mean"))
    #print("\n")
    
    
    House=pd.concat([House,c])
House["type"]="housekeeping"

In [15]:
Busco=pd.DataFrame()
for acc in ref["accession"].unique().tolist():
    
    clear_isos=ref.loc[(ref["accession"]==acc)&(ref["cat_assign"] == "one2one")]
    isos_type=augustus_refs(clear_isos).drop(columns=["ref_isoform","ref_gene"])
    busco_isos=isos_type.loc[isos_type["type"]=="busco"]

    
    b=fusion_types.loc[(fusion_types["busco"]==1)&(fusion_types["accession"]==acc)]["isoform"].tolist()
    fusion_isos=ref.loc[(ref["accession"]==acc)&(ref["isoform"].isin(b))]

    c=pd.concat([busco_isos,fusion_isos], ignore_index=True)
    #c=ref.loc[(ref["accession"]==f"{acc}") & (ref["gene"].isin(set(a[f"{acc}"].tolist() + b[f"{acc}"].tolist())))&~(ref["cat_assign"].isin(["antisense","novel"]))].copy()
    
    c["isoforms_per_gene"]= c.groupby(by="gene")["isoform"].transform(lambda x: len(x.tolist()))
    c["ORFs_per_gene"]= c.groupby(by=["gene"])["Sqanti_ORF_seq"].transform(lambda x: len(set(x.tolist())))
    c["isoforms_per_ORFs"]= c.groupby(by=["gene","Sqanti_ORF_seq"])["isoform"].transform(lambda x: len(x.tolist()))
    
    #print("Mean nlr isoforms per gene:")
    hipg=c[["gene","isoforms_per_gene"]].drop_duplicates()
    #print(hipg["isoforms_per_gene"].apply("mean"))

    #print("Mean nlr ORFs per gene:")
    hopg=c[["gene","ORFs_per_gene"]].drop_duplicates()
    #print(hopg["ORFs_per_gene"].apply("mean"))
    #print("\n")
    
    
    Busco=pd.concat([Busco,c])
Busco["type"]="busco"

In [16]:
Other=pd.DataFrame()
for acc in ref["accession"].unique().tolist():
    
    clear_isos=ref.loc[(ref["accession"]==acc)&(ref["cat_assign"] == "one2one")]
    isos_type=augustus_refs(clear_isos).drop(columns=["ref_isoform","ref_gene"])
    isos_type["type"] = isos_type["type"].fillna("other")
    other_isos=isos_type.loc[isos_type["type"]=="other"]

    
    b=fusion_types.loc[(fusion_types["other"]==1)&(fusion_types["accession"]==acc)]["isoform"].tolist()
    fusion_isos=ref.loc[(ref["accession"]==acc)&(ref["isoform"].isin(b))]

    c=pd.concat([other_isos,fusion_isos], ignore_index=True)
    
    c["isoforms_per_gene"]= c.groupby(by="gene")["isoform"].transform(lambda x: len(x.tolist()))
    c["ORFs_per_gene"]= c.groupby(by=["gene"])["Sqanti_ORF_seq"].transform(lambda x: len(set(x.tolist())))
    c["isoforms_per_ORFs"]= c.groupby(by=["gene","Sqanti_ORF_seq"])["isoform"].transform(lambda x: len(x.tolist()))
    
    #print("Mean nlr isoforms per gene:")
    hipg=c[["gene","isoforms_per_gene"]].drop_duplicates()
    #print(hipg["isoforms_per_gene"].apply("mean"))

    #print("Mean nlr ORFs per gene:")
    hopg=c[["gene","ORFs_per_gene"]].drop_duplicates()
    #print(hopg["ORFs_per_gene"].apply("mean"))
    #print("\n")
    
    
    Other=pd.concat([Other,c])
Other["type"]="other"

In [17]:
all_types=pd.concat([NLRs,Busco,House,Other], ignore_index=True)
all_types

,gene,isoform,accession,associated_gene,associated_transcript,exons,length,isoform_read_support,gene_read_support,structural_category,subcategory,Sqanti_ORF_seq,cat_assign,association,cat_read_support,gene_association_read_support,type,isoforms_per_gene,ORFs_per_gene,isoforms_per_ORFs
0,G1128,G1128.10,at6923,g1555,novel,1,3275,1,59,genic,mono-exon,MGNCVSLDVSCHHQTLNHACGCLFGDGNYIHMMEANLEALEKTIQE...,one2one,1,59,12,nlr,11,2,8.0
1,G1128,G1128.11,at6923,g1555,novel,1,2744,1,59,genic,mono-exon,MDGVLTFTVKMHDVIREMALWIASKFGKQKETLCVRSGAQLRQIPK...,one2one,1,59,12,nlr,11,2,3.0
2,G1128,G1128.12,at6923,g1555,novel,1,2795,1,59,genic,mono-exon,MDGVLTFTVKMHDVIREMALWIASKFGKQKETLCVRSGAQLRQIPK...,one2one,1,59,12,nlr,11,2,3.0
3,G1128,G1128.13,at6923,g1555,novel,1,1821,1,59,genic,mono-exon,MDGVLTFTVKMHDVIREMALWIASKFGKQKETLCVRSGAQLRQIPK...,one2one,1,59,12,nlr,11,2,3.0
4,G1128,G1128.2,at6923,g1555,novel,2,3331,1,59,novel_not_in_catalog,intron_retention,MGNCVSLDVSCHHQTLNHACGCLFGDGNYIHMMEANLEALEKTIQE...,one2one,1,59,12,nlr,11,2,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3419863,G16302,G16302.24,col0,ATCG01310_ATCG01300,novel,2,818,1,136,fusion,multi-exon,MAIHLYKTSTPSTRNGAVDSQVKSNPRNNLICGQHHCGKGRNARGI...,fusion,2,15,15,other,49,8,2.0
3419864,G16302,G16302.26,col0,ATCG01310_ATCG01300,novel,2,831,2,136,fusion,multi-exon,MAIHLYKTSTPSTRNGAVDSQVKSNPRNNLICGQHHCGKGRNARGI...,fusion,2,15,15,other,49,8,4.0
3419865,G16302,G16302.5,col0,ATCG01310_ATCG01300,novel,2,1204,1,136,fusion,multi-exon,MAIHLYKTSTPSTRNGAVDSQVKSNPRNNLICGQHHCGKGRNARGI...,fusion,2,15,15,other,49,8,4.0
3419866,G16302,G16302.7,col0,ATCG01310_ATCG01300,novel,2,1141,6,136,fusion,multi-exon,MAIHLYKTSTPSTRNGAVDSQVKSNPRNNLICGQHHCGKGRNARGI...,fusion,2,15,15,other,49,8,4.0


In [18]:
drop=[]
for i in tqdm(all_types.groupby("accession")):
    df = i[1]
    boolean=df["isoform"].value_counts()>1
    boolean=boolean.rename_axis('isoform').reset_index(name='bool')
    #print(boolean.loc[boolean["bool"]==True])
    #print(boolean)
    dups = boolean.loc[boolean["bool"]==True]["isoform"].tolist()
    for iso in dups:
        iso_df=df[df["isoform"]==iso]
        if "nlr" in iso_df["type"].tolist():
            drop.extend(iso_df[~(iso_df["type"]=="nlr")].index.tolist())
            continue
        if "housekeeping" in iso_df["type"].tolist():
            drop.extend(iso_df[~(iso_df["type"]=="housekeeping")].index.tolist())
            continue
        if "busco" in iso_df["type"].tolist():
            drop.extend(iso_df[~(iso_df["type"]=="busco")].index.tolist())
            continue

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [03:30<00:00, 11.72s/it]


In [19]:
all_types=all_types.drop(drop).drop_duplicates()

In [20]:
# This is the fall_typesdataframe to continue 
all_types
all_types.to_pickle("clear-fusion/clear-fusion_tama_augustus.pkl")
#all_types.to_pickle("../5_NLRs/clear-fusion_tama_augustus.pkl")
#all_types.to_pickle("../3_fusion_genes/clear-fusion_tama_augustus.pkl")